In [1]:
%loadFromPOM pom.xml

In [2]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;
import org.hl7.fhir.instance.model.api.*;
import java.nio.file.*;

In [3]:
FhirContext ctx = FhirContext.forR4();
ctx.getRestfulClientFactory().setSocketTimeout(60 * 10 * 100);
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

背景：FHIR API

In [4]:
// Load synthetic data from /synthetic-data/fhir directory
String path = "../synthetic-data/fhir";
try {
    Files.walk(Paths.get(path)).forEach(filePath -> {
        if (Files.isRegularFile(filePath)) {
           
            // Only load resources from the /synthetic-data/fhir directory with file name start with "practitionerinformation"
            if (filePath.toString().contains("practitionerInformation") || filePath.toString().contains("hospitalInformation")) {
                System.out.println("Loading " + filePath);
                // Read the file as a string
                String resourceString = "";
                try {
                    resourceString = new String(Files.readAllBytes(filePath));
                } catch (IOException e) {
                    e.printStackTrace();
                }
                // Parse the string to a FHIR resource
                Bundle bundle = ctx.newJsonParser().parseResource(Bundle.class, resourceString);
                // Use the client to store a new resource instance
                IBaseBundle outcome = client.transaction().withBundle(bundle).execute(); 
            }
        }
        {
            //System.out.println("Skipping " + filePath);
        }
    });
} catch (IOException e) {
    e.printStackTrace();
}

Loading ..\synthetic-data\fhir\hospitalInformation1702486193529.json
Loading ..\synthetic-data\fhir\practitionerInformation1702486193529.json


In [5]:
// Load synthetic data from /synthetic-data/fhir directory
String path = "../synthetic-data/fhir";
try {
    Files.walk(Paths.get(path)).forEach(filePath -> {
        if (Files.isRegularFile(filePath)) {
            try {
                String resource = Files.readString(filePath);
                Bundle bundle = ctx.newJsonParser().parseResource(Bundle.class, resource);
                for (Bundle.BundleEntryComponent entry : bundle.getEntry()) {
                    Resource r = entry.getResource();
                    // Print resource type and id
                    String type = r.getResourceType().name();
                    if(type.equals("Patient") == true ){
                        // Print patient name
                        Patient patient = (Patient) r;
                        System.out.println(patient.getName().get(0).getGivenAsSingleString() + " " + patient.getName().get(0).getFamily());
                    }
                }
                // create a client and post the transaction to the server
                IBaseBundle outcome = client.transaction().withBundle(bundle).execute();
                

            } catch (Exception e) {
                System.out.println("Error: " + filePath + " " + e.getMessage());
            }
        }
    });
} catch (IOException e) {
    System.out.println("Error reading files: " + e.getMessage());
}


Andre610 Lueilwitz711
Bao544 Gusikowski974
Carol737 Raymundo71 Lueilwitz711
Davina555 Jacqulyn231 MacGyver246
Frances376 Fahey393
Hans694 Ivory697 Bayer639
Jamey282 Moses679 Herzog843
Jarod376 Pedro316 Weissnat378
Kattie846 Cecile401 Tremblay80
Latosha740 Mercy752 Emmerich580
Mack300 Cory323 Stracke611
Magdalena964 Anita473 Núñez242
Trang668 Angie214 Abshire638
Warner493 Carter549
Warren653 Gaylord332 Hessel84
